In [21]:
import pandas as pd
import configparser
import openai

# Read credentials
parser = configparser.ConfigParser()
parser.read('./config/config.ini')
print(parser['openai']['TRAINING_DATA_FILE_ID'])

# Read raw dataset
# Change the iteration folder number for every iteration
data = pd.read_csv("./dataset/raw-data.csv")
data_df = pd.DataFrame(data)

data_df

file-zlE2ur7f6Kghn0MtMgIFT8qC


,prompt,completion
0,hi there,0
1,hey,0
2,hello,0
3,good morning,0
4,good afternoon,0
...,...,...
527,teach me how to use,6
528,sikhao kaise use karein,6
529,sahaayta,6
530,sahaayta kariye,6


In [3]:
# Convent CSV to JSONL
data_df.to_json("./dataset/training-data.jsonl", orient='records', lines=True)

In [22]:
# Add OpenAI credentials
openai.api_key = parser['openai']['ACCESS_KEY']

In [34]:
# Lists the fine tune odels created
openai.FineTune.list()

<OpenAIObject list at 0x7ff1fa82de40> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine-tune",
      "id": "ft-O7RMjF3IiBatt5YDf2YY2xqv",
      "hyperparams": {
        "n_epochs": 4,
        "batch_size": null,
        "prompt_loss_weight": 0.01,
        "classification_n_classes": 7,
        "learning_rate_multiplier": null,
        "compute_classification_metrics": true
      },
      "organization_id": "org-gyLMiLCwRbX042wASGsJYdqq",
      "model": "ada",
      "training_files": [
        {
          "object": "file",
          "id": "file-zlE2ur7f6Kghn0MtMgIFT8qC",
          "purpose": "fine-tune",
          "filename": "file",
          "bytes": 24607,
          "created_at": 1691117915,
          "status": "processed",
          "status_details": null
        }
      ],
      "validation_files": [
        {
          "object": "file",
          "id": "file-cTkPKbSgHtpWTqlwc2vkfQvM",
          "purpose": "fine-tune",
          "filename": "file",
          "byte

In [15]:
# Sublets the dataset into training and validation jsonl
# Formats the dataset for best training results
# Run in terminal

cd <project folder>/packages/classifer/dataset
openai tools fine_tunes.prepare_data -f 'training-data.jsonl' -q

SyntaxError: invalid syntax (3277997256.py, line 5)

In [10]:
# Upload training and validation jsonl file
# Retrieve the ID
# Run this code once to upload the file

training_data = openai.File.create(
  file=open("./dataset/training-data_prepared_train.jsonl", "rb"),
  purpose='fine-tune'
)
training_data_file_id = training_data.id

validation_data = openai.File.create(
  file=open("./dataset/training-data_prepared_valid.jsonl", "rb"),
  purpose='fine-tune'
)
validation_data_file_id = validation_data.id

In [11]:
print(training_data_file_id)
print(validation_data_file_id)

file-zlE2ur7f6Kghn0MtMgIFT8qC
file-cTkPKbSgHtpWTqlwc2vkfQvM


In [20]:
# Snippet to delete the file

training_data_file_id = parser['openai']['TRAINING_DATA_FILE_ID']
validation_data_file_id = parser['openai']['VALIDATION_DATA_FILE_ID']

openai.File.delete(training_data_file_id)
openai.File.delete(validation_data_file_id)

In [23]:
# Equivalent CLI query: openai api fine_tunes.create -t "training-data_prepared_train.jsonl" -v "training-data_prepared_valid.jsonl" --compute_classification_metrics --classification_n_classes 7

openai.FineTune.create(
  training_file=parser['openai']['TRAINING_DATA_FILE_ID'],
  validation_file=parser['openai']['VALIDATION_DATA_FILE_ID'],
  model=parser['openai']['MODEL_NAME'],
  compute_classification_metrics=True,
  classification_n_classes=7,
  suffix="ayra"
)

<FineTune fine-tune id=ft-O7RMjF3IiBatt5YDf2YY2xqv at 0x7ff1fa704900> JSON: {
  "object": "fine-tune",
  "id": "ft-O7RMjF3IiBatt5YDf2YY2xqv",
  "hyperparams": {
    "n_epochs": 4,
    "batch_size": null,
    "prompt_loss_weight": 0.01,
    "classification_n_classes": 7,
    "learning_rate_multiplier": null,
    "compute_classification_metrics": true
  },
  "organization_id": "org-gyLMiLCwRbX042wASGsJYdqq",
  "model": "ada",
  "training_files": [
    {
      "object": "file",
      "id": "file-zlE2ur7f6Kghn0MtMgIFT8qC",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 24607,
      "created_at": 1691117915,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [
    {
      "object": "file",
      "id": "file-cTkPKbSgHtpWTqlwc2vkfQvM",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 6062,
      "created_at": 1691117916,
      "status": "processed",
      "status_details": null
    }
  ],
  "result_files": [

In [43]:
openai.FineTune.retrieve(id='ft-O7RMjF3IiBatt5YDf2YY2xqv')

<FineTune fine-tune id=ft-O7RMjF3IiBatt5YDf2YY2xqv at 0x7ff1fa779cb0> JSON: {
  "object": "fine-tune",
  "id": "ft-O7RMjF3IiBatt5YDf2YY2xqv",
  "hyperparams": {
    "n_epochs": 4,
    "batch_size": null,
    "prompt_loss_weight": 0.01,
    "classification_n_classes": 7,
    "learning_rate_multiplier": null,
    "compute_classification_metrics": true
  },
  "organization_id": "org-gyLMiLCwRbX042wASGsJYdqq",
  "model": "ada",
  "training_files": [
    {
      "object": "file",
      "id": "file-zlE2ur7f6Kghn0MtMgIFT8qC",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 24607,
      "created_at": 1691117915,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [
    {
      "object": "file",
      "id": "file-cTkPKbSgHtpWTqlwc2vkfQvM",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 6062,
      "created_at": 1691117916,
      "status": "processed",
      "status_details": null
    }
  ],
  "result_files": [

In [44]:
# Checks the status of the fine tune job

openai.FineTune.list_events(id="ft-O7RMjF3IiBatt5YDf2YY2xqv")

# Command Line version

# openai --api-key <api-key> api fine_tunes.follow -i <fine-tune_id>

<OpenAIObject list at 0x7ff2342558a0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine-tune-event",
      "level": "info",
      "message": "Created fine-tune: ft-O7RMjF3IiBatt5YDf2YY2xqv",
      "created_at": 1691118281
    }
  ]
}